In [727]:
from gym_go.envs_copy import go_env
from gym_go import gogame
import torch 
from math import sqrt
import time



In [728]:
board_size=7
komi=0
l2_loss = 1
c_puct = 5
N_threshold = 10
temperature_MCTS = 2
N_tree_search = 100
qu_ration = 0.5
epsilone = 0.0001

# 9*9 stats
# roll_time = 0.285

# 7*7 stats
# roll_time = 0.11

In [729]:
env = go_env.GoEnv(size=board_size, komi=komi)
state, reward, done, info = env.step((0,0))
print(state[:4], '\n----\n')
state, reward, done, info = env.step((1,0))
print(state[:4])


[[[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]

 [[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]] 
----

[[[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0.

In [730]:
class OutputFormating(torch.Tensor):
    @property
    def value(self):
        """Return the last element along dim=-1"""
        if self.ndim == 1:
            return self[-1]
        else:
            return self[..., -1]

    @property
    def result(self):
        """Return all elements except the last along dim=-1"""
        if self.ndim == 1:
            return self[:-1]
        else:
            return self[..., :-1]


In [731]:
class GoBot(torch.nn.Module):
  """
  See only the bord as playing as the black player 
  (so when it's white we just exchange the black and white color so it can see it as black)
  """
  def __init__(self):
    super(GoBot, self).__init__()

    # input channel = 1, output channels = 6, kernel size = 5
    # input image size = (32, 32), image output size = (28, 28)
    self.convol1 = torch.nn.Conv2d(4, 20, 7, padding=3, bias=True)
    
    self.convol2 = torch.nn.Conv2d(20, 40, 5, padding=2, bias=True)

    self.linear1 = torch.nn.Linear(40*(board_size**2), 150, bias=True)

    # input dim = 120, output dim = 84
    self.linear2 = torch.nn.Linear(150, 110, bias=True) 

    self.linear3 = torch.nn.Linear(110, board_size**2 + 2, bias=True) # 83 pour les 81 case possible le suivant pass et le dernier pour la valeur


  def forward(self, x):
    if not isinstance(x, torch.Tensor):
      x = torch.tensor(x, dtype=torch.float32)

    if x.ndim == 3: 
        x = x.unsqueeze(0)

    if x[:,2,0,0] == 1:
      x[:, [0,1], ...] = x[:, [1,0], ...]
    
    x = x[:,[0,1,3,4],...]


    x = torch.nn.ReLU() (self.convol1(x))

    x = torch.nn.ReLU() (self.convol2(x))

    # flatten the feature maps into a long vector
    x = x.view(x.shape[0], -1) 

    x = torch.nn.ReLU() (self.linear1(x))
    x = torch.nn.ReLU() (self.linear2(x))
    x = self.linear3(x)
    

    return OutputFormating(x)
  
  @property
  def value(self):
    return 

In [732]:
net = GoBot()
out = net.forward(state)
print(out.result[0])
print(out.result[0,10])



OutputFormating([-0.0489,  0.0757, -0.0445,  0.0699, -0.0273,  0.0226,  0.0341,
                  0.0289, -0.0040, -0.0984,  0.0565, -0.0251, -0.0506, -0.0877,
                  0.0143, -0.0609, -0.0930,  0.1133,  0.0702,  0.1191,  0.0457,
                  0.0040, -0.0841, -0.0584,  0.0612, -0.0775,  0.0550,  0.0460,
                 -0.0260,  0.0330,  0.1022, -0.0308,  0.0442, -0.0078, -0.0046,
                  0.0246,  0.0556, -0.0624, -0.0668,  0.0463, -0.0303,  0.0479,
                  0.0807, -0.0888,  0.1031,  0.0872, -0.0010,  0.0763, -0.0329,
                 -0.0259], grad_fn=<AliasBackward0>)
OutputFormating(0.0565, grad_fn=<AliasBackward0>)


In [733]:
def compute_loss(output, target, reward):
  value_loss = output.value - reward
  policy_loss = torch.dot(target, output.result)
  regulation = l2_loss
  return value_loss + policy_loss + regulation

In [734]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

In [735]:
def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)

      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(net, data_loader, optimizer, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.


  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (state, targets, reward) in enumerate(data_loader):
    # Load data into GPU
    state = state.to(device)
    targets = targets.to(device)

    # Forward pass
    outputs = net(state)

    # Apply the loss
    loss = compute_loss(outputs, targets, reward)

    # Reset the optimizer

    # Backward pass
    loss.backward()

    # Update parameters
    optimizer.step()

    optimizer.zero_grad()

    # Better print something, no?
    samples+=state.shape[0]
    cumulative_loss += loss.item()
    # _, predicted = outputs[:82].max(1)   # for the action which has been remembered

  return cumulative_loss/samples

In [736]:
def roll_policy(state, net):    
    """
    return :
        the score of the outcome of the self played game seen by the first player to play
    """
    player = state[2,0,0]
    game_ended = gogame.game_ended(state)

    while not game_ended:
        if torch.all(torch.tensor(state[3,:,:]) == 1):
            action = board_size**2
        else :
            result = net([state]).result[0]
            sorted_indices_actions = sorted(range(len(result)), key=lambda i: result[i])
            invalid_moves = gogame.invalid_moves(state)
            for i in range(board_size**2+1) :
                action = sorted_indices_actions[i]
                if not invalid_moves[action] : 
                    break
  
        state = gogame.next_state(state, action)
        game_ended = gogame.game_ended(state)

    score = gogame.winning(state, komi)  # from absolut black perspective
    if player :
        score = - score
    return score        # from the perspective of the player who where supposed to play at the starting state of the roll_policy
    

In [737]:
state = gogame.init_state(board_size)
print(roll_policy(state, net))

1.0


In [738]:
net = GoBot()


class MCTS():
    def __init__(self, state, action : int, p : torch.Tensor, depth = 0):
        self.next_nodes = []
        self.pass_node = None
        self.state = state
        self.action = action
        self.N = 0
        self.Wv = 0
        self.Wr = 0
        self.Q = 0
        self.p = p
        self.depth = depth

    def next_state(self):
        if self.action == None :
            return self.state
        else :
            return gogame.next_state(self.state, self.action)
    
    def action_2d(self):
        if self.action == None :
            return "None"
        else :
            return self.action // board_size, self.action % board_size


    def exctend_tree(self):
        next_state = self.next_state()
        result = net([next_state]).result[0]
        for i in range(board_size):
            for j in range(board_size):
                if next_state[3,i,j] == 1:
                    p = 0
                else : 
                    p = max(epsilone,result[i*board_size + j])
                action = i*board_size + j
                # adding the mouve at possition (i,j)
                self.next_nodes.append(
                    MCTS(next_state, 
                        action, 
                        p,
                        self.depth + 1))
        # adding the pass mouve
        self.next_nodes.append(
            MCTS(next_state, 
                board_size**2, 
                epsilone,
                self.depth + 1))


    def push_search(self):
        if self.N >= N_threshold :
            if not self.next_nodes :
                print("extende : ", self.action_2d(), self.depth)
                self.exctend_tree()

            max_score = 0      # all scores can be <0 so can be improved here (maybe take the score of the default pass mouve)
            # max_score = self.next_nodes[-1].Q + c_puct * self.next_nodes[-1].p * sqrt(self.N)/(1+self.next_nodes[-1].N)
            node_to_search = self.next_nodes[-1]    # default next mouve is the pass mouve (used if no mouves has a positive score)
            for next_node in self.next_nodes :
                score = next_node.Q + c_puct * next_node.p * sqrt(self.N)/(1+next_node.N)
                # print(score, next_node)
                if score > max_score :
                    max_score = score
                    node_to_search = next_node
            value, output = node_to_search.push_search()
            

        else :
            # print(self.action)
            # print(self.state[3,:,:])
            next_state = self.next_state()
            value = net(next_state).value
            output = roll_policy(next_state, net)
            if self.depth % 2 == 1 :
                value = - value
                output = - output

        self.N += 1
        self.Wv += value
        self.Wr += output
        self.Q = ((1-qu_ration) * self.Wv + qu_ration * self.Wr)/self.N
        return value, output
    
    def best_next_node(self):
        max_visited_N = 0
        i = 0
        for next_node in self.next_nodes :
            i += 1
            visited_N = next_node.N
            if visited_N > max_visited_N :
                max_visited_N = visited_N
                best_node = next_node
        return best_node
    
    @staticmethod
    def tree_policy(node):
        actions = []
        visited_N = []
        for next_node in node.next_nodes :
            actions.append(next_node.action)
            visited_N.append(next_node.N)
        diviseur = sum(N**(1/temperature_MCTS) for N in visited_N)
        sorted_index = sorted(range(len(actions)), key=lambda i : actions[i])
        MCTS_policy = [visited_N[i]/diviseur for i in sorted_index]
        return MCTS_policy
    
    def __str__(self):
        print("___ node info ___")
        print("action : {} | depth : {}".format(self.action_2d(), self.depth))
        print(self.N)
        # print(self.state[3])
        return ' '
        


In [739]:
state = gogame.init_state(board_size)
root = MCTS(state, None, 1)
for i in range(N_threshold + 3) :
    root.push_search()


extende :  None 0


In [740]:
def sample_one_self_play_MCTS(net):
    data_set = []
    state = gogame.init_state(board_size)
    root = MCTS(state, None, 1)
    while not gogame.game_ended(state) : 
        print(root)
        tmp = time.time()
        
        for i in range(N_tree_search) : 
            root.push_search()
        MCTS_policy = MCTS.tree_policy(root)
        data_set.append((state,MCTS_policy))
    
        next_root = root.best_next_node()
        next_state = gogame.next_state(state, next_root.action)
        root = next_root
        state = next_state
        print("\n", time.time() - tmp)
    

In [ ]:
root = sample_one_self_play_MCTS(net)

___ node info ___
action : None | depth : 0
0
 
extende :  None 0


In [ ]:
def sample_self_play_MCTS_batchs(N, net, batch_size):
  for i in range(N):
    sample_one_self_play_MCTS(net)

#   training_data = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
  training_data = []

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)

  return train_loader

In [ ]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
'''

def main(batch_size=128,
         device='cuda:0',
         learning_rate=0.01,
         weight_decay=0.000001,
         momentum=0.9,
         epochs=50,
         K_early_stopping = 5):

  train_loader = sample_data(batch_size, net)

  net = GoBot()

  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, device)


  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('-----------------------------------------------------')

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, device)
    
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, device)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('-----------------------------------------------------')
  return net